# Analyse Exploratoire du fichier myreport_sql_gestionairedememoire_full.csv

### Imports (fichier & modules)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
#import os
#print(os.getcwd())
#print(os.listdir('241016_LogETL'))

In [ ]:
tab = pd.read_csv('../../data/logServer/myreport_sql_gestionairedememoire_full.csv', sep=',')

### Le fichier myreport_sql_gestionairedememoire_full.csv : taille, colonnes

In [ ]:
print("il y a ",tab.shape[0]," lignes et ",tab.shape[1]," colonnes dans le data frame")

In [ ]:
tab.columns

In [ ]:
tab.info()

On peut déjà voir que :
- faudra supprimer les colonnes 8 et 9 car elles sont vides
- colonnes 21, 23, 25, 26 contiennent certains élements vides

In [ ]:
# Calculer le nombre de valeurs uniques pour chaque colonne
nb_valeurs_uniques = tab.nunique()

# Afficher les résultats
print(nb_valeurs_uniques)

En plus des colonnes 8 et 9, on pourra supprimer les colonnes 15, 16, 17. 

In [ ]:
tab.head()


### Transformation des dates

In [ ]:
# Convertir toutes les dates de la colonne 'Date et heure(RAW)'
tab['Date et heure'] = tab["Date et heure"].str.split(" - ").str[0]

tab=tab.iloc[:-5]
# Si vous avez un format comme "JJ/MM/AAAA HH:MM:SS", vous pouvez préciser :
tab['Date et heure'] = pd.to_datetime(tab['Date et heure'], format="%d/%m/%Y %H:%M:%S")

In [ ]:
tab.head()


In [ ]:
start_date = '2024-08-24 23:32:03'
end_date = '2024-10-15 23:31:49'
tab = tab[(tab['Date et heure'] >= start_date) & (tab['Date et heure'] <= end_date)]

In [ ]:
tab.shape

### Réagencement et Suppression des colonnes

In [ ]:
 # suppression des colonnes inutiles
tab = tab.drop(columns=["Date et heure(RAW)"])
tab = tab.drop(columns=["Mémoire de connexion"])
tab = tab.drop(columns=["Mémoire de l'optimiseur"])
tab = tab.drop(columns=["Mémoire totale du serveur"])
tab = tab.drop(columns=["Mémoire du serveur cible"])
tab = tab.drop(columns=["Mémoire du cache SQL"])

tab = tab.drop(columns=["Temps mort"])
tab = tab.drop(columns=["Couverture"])

# renommage des colonnes
tab.rename(columns={"Mémoire de connexion(RAW)": "Mémoire de connexion_sql_gestmemoire(Ko)"}, inplace=True)
tab.rename(columns={"Mémoire de l'optimiseur(RAW)": "Mémoire de l'optimiseur_sql_gestmemoire(Ko)"}, inplace=True)
tab.rename(columns={"Mémoire totale du serveur(RAW)": "Mémoire totale du serveur_sql_gestmemoire(Ko)"}, inplace=True)
tab.rename(columns={"Mémoire du serveur cible(RAW)": "Mémoire du serveur cible_sql_gestmemoire(Ko)"}, inplace=True)
tab.rename(columns={"Mémoire du cache SQL(RAW)": "Mémoire du cache SQL_sql_gestmemoire(Ko)"}, inplace=True)

tab.rename(columns={"Temps mort(RAW)": "Temps mort_sql_gestmemoire(%)"}, inplace=True)
tab.rename(columns={"Couverture(RAW)": "Couverture_sql_gestmemoire(%)"}, inplace=True)

### Analyse temporelle

In [ ]:
import matplotlib.pyplot as plt
import itertools
import pandas as pd

import matplotlib.pyplot as plt

# Vérifier que 'date_full' est bien au format datetime
tab["Date et heure"] = pd.to_datetime(tab["Date et heure"])

# Liste des colonnes à exclure
excluded_cols = ["date_full", "date", "heure", "Date et heure(RAW)"]

# Tracer chaque colonne en fonction du temps
for col in tab.columns:
    if col not in excluded_cols:  # Exclure les colonnes non pertinentes
        plt.figure(figsize=(10, 5))

        plt.plot(tab["Date et heure"], tab[col], marker='o', linestyle='-')

        plt.xlabel("Date et heure")
        plt.ylabel(f"{col}")  # Correction de la syntaxe
        plt.title(f"Évolution de {col} en fonction du temps")
        plt.legend([col])  # Correction de la légende
        plt.xticks(rotation=45)
        plt.grid()

        plt.show()

In [ ]:
import seaborn as sns

# Exclure les colonnes non numériques
numerical_cols = tab.select_dtypes(include=['number']).columns

# Tracer un boxplot pour chaque colonne
for col in numerical_cols:
    if col not in excluded_cols:  # Exclure les colonnes non pertinentes
        plt.figure(figsize=(8, 5))
        sns.boxplot(y=tab[col])
        plt.title(f"Boxplot de {col}")
        plt.grid()
        plt.show()

In [ ]:
# Sélection des colonnes numériques
numerical_cols = tab.select_dtypes(include=['number']).columns

# Filtre pour exclure les colonnes non pertinentes
cols_to_plot = [col for col in numerical_cols if col not in excluded_cols]

rows=(math.ceil(len(cols_to_plot)/3))
# Créer une figure avec 2 lignes et 3 colonnes de sous-graphiques (axes)
fig, axes = plt.subplots(nrows=rows, ncols=3, figsize=(15, rows*5))

# Convertir "axes" en un tableau 1D pour plus de flexibilité
axes = axes.flatten()

# Boucler sur les colonnes et tracer un boxplot dans chaque sous-graphique
for i, col in enumerate(cols_to_plot):
    sns.boxplot(y=tab[col], ax=axes[i])
    axes[i].set_title(f"Boxplot de {col}")
    axes[i].grid()

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher la figure
plt.show()

### drop des colonnes inutiles

In [ ]:
tab = tab.drop(columns=["Temps mort_sql_gestmemoire(%)"])
tab = tab.drop(columns=["Couverture_sql_gestmemoire(%)"])
tab = tab.drop(columns=["Mémoire du serveur cible_sql_gestmemoire(Ko)"])

### Enregistrement du nouveau dataframe dans un nouveau csv

In [ ]:
#### ATTENTION au path et nom de fichier
tab.to_csv("../data/logServer/filtered/myreport_sql_gestionairedememoire_filtered.csv", index=False, encoding='utf-8')